In [1]:
import pandas

def read_data(file_name):
    csv_file = pandas.read_csv(f'../data/{file_name}.csv')
    return csv_file


student_info = read_data('studentInfo')
# convert type of student
# student_info['id_student'] = student_info['id_student'].astype(str)

student_info['id'] = student_info['id_student'].astype(str) + '_' + student_info['code_module'] + '_' + student_info["code_presentation"]
student_info.head()
student_info.describe(include='all')

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,id
count,32593,32593,3.259300e+04,32593,32593,32593,31482,32593,32593.000000,32593.000000,32593,32593,32593
unique,7,4,NaN,2,13,5,10,3,NaN,NaN,2,4,32593
top,BBB,2014J,NaN,M,Scotland,A Level or Equivalent,20-30%,0-35,NaN,NaN,N,Pass,11391_AAA_2013J
freq,7909,11260,NaN,17875,3446,14045,3654,22944,NaN,NaN,29429,12361,1
mean,NaN,NaN,7.066877e+05,NaN,NaN,NaN,NaN,NaN,0.163225,79.758691,NaN,NaN,NaN
std,NaN,NaN,5.491673e+05,NaN,NaN,NaN,NaN,NaN,0.479758,41.071900,NaN,NaN,NaN
min,NaN,NaN,3.733000e+03,NaN,NaN,NaN,NaN,NaN,0.000000,30.000000,NaN,NaN,NaN
25%,NaN,NaN,5.085730e+05,NaN,NaN,NaN,NaN,NaN,0.000000,60.000000,NaN,NaN,NaN
50%,NaN,NaN,5.903100e+05,NaN,NaN,NaN,NaN,NaN,0.000000,60.000000,NaN,NaN,NaN
75%,NaN,NaN,6.444530e+05,NaN,NaN,NaN,NaN,NaN,0.000000,120.000000,NaN,NaN,NaN


In [3]:
student_registration = read_data("studentRegistration")
merged_df = pandas.merge(student_info, student_registration, on=['id_student', 'code_module', 'code_presentation'])
merged_df.head()
merged_df.describe(include='all')

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,id,date_registration,date_unregistration
count,32593,32593,3.259300e+04,32593,32593,32593,31482,32593,32593.000000,32593.000000,32593,32593,32593,32548.000000,10072.000000
unique,7,4,NaN,2,13,5,10,3,NaN,NaN,2,4,32593,NaN,NaN
top,BBB,2014J,NaN,M,Scotland,A Level or Equivalent,20-30%,0-35,NaN,NaN,N,Pass,11391_AAA_2013J,NaN,NaN
freq,7909,11260,NaN,17875,3446,14045,3654,22944,NaN,NaN,29429,12361,1,NaN,NaN
mean,NaN,NaN,7.066877e+05,NaN,NaN,NaN,NaN,NaN,0.163225,79.758691,NaN,NaN,NaN,-69.411300,49.757645
std,NaN,NaN,5.491673e+05,NaN,NaN,NaN,NaN,NaN,0.479758,41.071900,NaN,NaN,NaN,49.260522,82.460890
min,NaN,NaN,3.733000e+03,NaN,NaN,NaN,NaN,NaN,0.000000,30.000000,NaN,NaN,NaN,-322.000000,-365.000000
25%,NaN,NaN,5.085730e+05,NaN,NaN,NaN,NaN,NaN,0.000000,60.000000,NaN,NaN,NaN,-100.000000,-2.000000
50%,NaN,NaN,5.903100e+05,NaN,NaN,NaN,NaN,NaN,0.000000,60.000000,NaN,NaN,NaN,-57.000000,27.000000
75%,NaN,NaN,6.444530e+05,NaN,NaN,NaN,NaN,NaN,0.000000,120.000000,NaN,NaN,NaN,-29.000000,109.000000


In [5]:
courses = read_data("courses")
merged_df = pandas.merge(merged_df, courses, on=['code_module', 'code_presentation'])
merged_df.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,id,date_registration,date_unregistration,module_presentation_length
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,11391_AAA_2013J,-159.0,NaN,268
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,28400_AAA_2013J,-53.0,NaN,268
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn,30268_AAA_2013J,-92.0,12.0,268
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,31604_AAA_2013J,-52.0,NaN,268
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,32885_AAA_2013J,-176.0,NaN,268
